# Analisis Anomali Network Menggunakan FP-Growth dan Association Rules

## Deskripsi Proyek
Notebook ini menganalisis log jaringan untuk mendeteksi anomali menggunakan algoritma **FP-Growth** (Frequent Pattern Growth) dan **Association Rules**. Tujuan utama adalah menemukan pola hubungan antara kondisi jaringan dan diagnosis anomali.

## Struktur Analisis
1. **Encoding Data**: Mengonversi data log menjadi format transaksi yang dapat dianalisis
2. **FP-Growth Algorithm**: Menemukan pola frekuen dengan threshold minimum support
3. **Association Rules**: Menghasilkan aturan asosiasi untuk prediksi diagnosis
4. **Filtering & Visualization**: Menyaring aturan yang relevan dan menampilkan hasil akhir

## Tahap 1: Import Library dan Persiapan Data

Pada tahap ini, kami mengimpor library yang diperlukan untuk analisis frequent pattern mining dan memuat dataset yang sudah dibersihkan dari tahap preprocessing.

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules
import pandas as pd

# Import cleaned data dari hasil preprocessing
df_cleaned = pd.read_csv('../Data/Data_Siap_Mining.csv')
print(f"Dataset berhasil dimuat: {df_cleaned.shape[0]} baris, {df_cleaned.shape[1]} kolom")

## Tahap 2: Encoding Data dan Perhitungan FP-Growth

### Proses Encoding
- **TransactionEncoder**: Mengkonversi data log dari format text menjadi format transaksi (one-hot encoded)
- Setiap item dalam 'items' column diubah menjadi kolom boolean terpisah
- Output: DataFrame `df_encoded` dengan nilai True/False untuk setiap atribut

### Perhitungan Support
- **Minimum Support**: Threshold minimum untuk pola yang dianggap "sering" terjadi
- **MIN_SUPPORT_COUNT = 20**: Pola harus muncul minimal 20 kali dalam dataset
- **min_support_pct**: Persentase support dihitung dari total jumlah transaksi

### Algoritma FP-Growth
- **fpgrowth**: Algoritma efisien untuk mining pola frekuen
- Lebih cepat dari Apriori karena menggunakan FP-tree structure
- Output: `frequent_itemsets` berisi semua pola yang memenuhi threshold

### Association Rules
- Menghasilkan aturan IF-THEN dari pola frekuen
- **min_threshold = 0.6**: Hanya aturan dengan confidence ≥ 60% yang dipertahankan
- Metrik yang dihitung: support, confidence, lift, dsb.

In [ ]:
# ===== STEP 1: ENCODING DATA =====
# Inisialisasi TransactionEncoder
te = TransactionEncoder()

# Konversi kolom 'items' dari string menjadi list dan lakukan fit-transform
items_list = df_cleaned['items'].apply(lambda x: eval(x)).tolist()
te_ary = te.fit(items_list).transform(items_list)

# Buat DataFrame dengan kolom yang sesuai
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)
print(f"\n✓ Data berhasil di-encode:")
print(f"  - Shape: {df_encoded.shape}")
print(f"  - Atribut unik: {len(df_encoded.columns)}")
print(f"\nSample data (5 baris pertama):\n{df_encoded.head()}")

# ===== STEP 2: FP-GROWTH ALGORITHM =====
# Hitung minimum support percentage
MIN_SUPPORT_COUNT = 20
min_support_pct = MIN_SUPPORT_COUNT / len(df_encoded)
print(f"\n✓ Konfigurasi Minimum Support:")
print(f"  - MIN_SUPPORT_COUNT: {MIN_SUPPORT_COUNT}")
print(f"  - Total Transaksi: {len(df_encoded)}")
print(f"  - min_support_pct: {min_support_pct:.4f} ({min_support_pct*100:.2f}%)")

# Jalankan FP-Growth
frequent_itemsets = fpgrowth(df_encoded, min_support=min_support_pct, use_colnames=True)
print(f"\n✓ FP-Growth selesai:")
print(f"  - Pola frekuen ditemukan: {len(frequent_itemsets)}")
print(f"\nTop 5 frequent itemsets:\n{frequent_itemsets.sort_values('support', ascending=False).head()}")

# ===== STEP 3: GENERATE ASSOCIATION RULES =====
MIN_CONFIDENCE = 0.6
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=MIN_CONFIDENCE)
print(f"\n✓ Association Rules berhasil dibuat:")
print(f"  - MIN_CONFIDENCE: {MIN_CONFIDENCE}")
print(f"  - Jumlah rules yang dihasilkan: {len(rules)}")
print(f"\nMetrik yang tersedia: {list(rules.columns)}")

## Tahap 3: Penyaringan dan Tampilan Hasil Akhir

### Konsep Filtering
Dari semua association rules yang dihasilkan, kita hanya ingin menyimpan rules yang:
- **Consequent (Kesimpulan)**: Harus berupa diagnosis/label anomali
  - NORMAL: Kondisi jaringan normal
  - UPSTREAM_FAILURE: Kegagalan uplink
  - LINK_FAILURE: Kegagalan link
  - DDOS_ATTACK: Serangan DDoS
  - BROADCAST_STORM: Broadcast storm

Contoh Rule yang BENAR (yang kita inginkan):
- IF `ether1=down AND queue=high` THEN `LINK_FAILURE` ✓

Contoh Rule yang SALAH (yang kita buang):
- IF `LINK_FAILURE` THEN `ether1=down` ✗

### Metrik Evaluasi
- **Support**: Seberapa sering pola ini muncul dalam dataset (0-1)
- **Confidence**: Probabilitas consequent terjadi jika antecedent terjadi (0-1)
- **Lift**: Seberapa kuat hubungan antara antecedent dan consequent (>1 = hubungan positif)

In [ ]:
# ===== STEP 4: FILTERING RULES =====
# Definisikan label diagnosis yang valid
DIAGNOSIS_LABELS = ['NORMAL', 'UPSTREAM_FAILURE', 'LINK_FAILURE', 'DDOS_ATTACK', 'BROADCAST_STORM']

# Fungsi untuk mengecek apakah consequent mengandung diagnosis
def is_diagnosis(consequents):
    """
    Cek apakah consequents (frozenset) mengandung minimal 1 label diagnosis
    """
    return any(label in consequents for label in DIAGNOSIS_LABELS)

# Filter rules - hanya ambil yang berakhir dengan diagnosis
final_rules = rules[rules['consequents'].apply(is_diagnosis)].copy()
print(f"✓ Rules setelah filtering:")
print(f"  - Total rules sebelum filter: {len(rules)}")
print(f"  - Rules dengan diagnosis consequent: {len(final_rules)}")
print(f"  - Rules yang dihilangkan: {len(rules) - len(final_rules)}")

# ===== STEP 5: SORTING =====
# Urutkan berdasarkan confidence (tertinggi) kemudian support (tertinggi)
final_rules = final_rules.sort_values(['confidence', 'support'], ascending=[False, False])
print(f"\n✓ Rules sudah diurutkan berdasarkan confidence dan support (descending)")

# ===== STEP 6: DISPLAY HASIL =====
# Pilih kolom yang akan ditampilkan
final_rules_display = final_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

print(f"\n{'='*100}")
print(f"TOP 10 ASSOCIATION RULES - NETWORK ANOMALY DETECTION")
print(f"{'='*100}\n")

# Tampilkan dengan format yang lebih rapi
for idx, (i, row) in enumerate(final_rules_display.head(10).iterrows(), 1):
    antecedents = ', '.join(list(row['antecedents']))
    consequents = ', '.join(list(row['consequents']))
    
    print(f"[Rule {idx}]")
    print(f"  IF:   {antecedents}")
    print(f"  THEN: {consequents}")
    print(f"  Support: {row['support']:.4f} | Confidence: {row['confidence']:.4f} | Lift: {row['lift']:.4f}")
    print()

# ===== STEP 7: SAVE HASIL =====
# Simpan top 10 rules ke CSV
final_rules_display.head(10).to_csv('../Final_Skripsi_Rules.csv', index=False)
print(f"{'='*100}")
print(f"✓ Top 10 rules berhasil disimpan ke 'Final_Skripsi_Rules.csv'")
print(f"{'='*100}")

## Kesimpulan dan Interpretasi

### Hasil Analisis
Notebook ini menghasilkan association rules yang menunjukkan hubungan antara kondisi jaringan dan tipe anomali. Setiap rule memiliki:
- **Antecedents (IF)**: Kondisi atau kombinasi kondisi jaringan yang diamati
- **Consequents (THEN)**: Diagnosis/prediksi tipe anomali yang kemungkinan terjadi
- **Confidence**: Tingkat akurasi prediksi (semakin tinggi semakin baik)
- **Support**: Seberapa sering pola ini muncul dalam data (frekuensi)
- **Lift**: Seberapa kuat hubungan antara kondisi dan diagnosis

### Penggunaan Praktis
Hasil rules ini dapat digunakan untuk:
1. **Early Warning System**: Mendeteksi anomali sebelum terjadi dengan mengecek kondisi antecedent
2. **Root Cause Analysis**: Memahami faktor-faktor apa yang menyebabkan setiap tipe anomali
3. **Network Optimization**: Meningkatkan monitoring dan maintenance protokol berdasarkan insights
4. **Decision Support**: Membantu network administrator membuat keputusan yang lebih cepat dan tepat

### File Output
- **Final_Skripsi_Rules.csv**: Berisi top 10 rules dalam format CSV yang dapat digunakan untuk proses selanjutnya